In [1]:
import torch
from torch import nn, optim
import os 
import torch.nn.functional as F

In [2]:
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
encoder_weights = Path("encoder/saved_models/pretrained.pt")
encoder.load_model(encoder_weights)
vocoder_weights = Path("vocoder/saved_models/pretrained/pretrained.pt")
syn_dir = Path("synthesizer/saved_models/logs-pretrained/taco_pretrained")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)

D:\pycharm_project\vs2ttsvoice\encoder\audio.py:13: UserWarning: Unable to import 'webrtcvad'. This package enables noise removal and is recommended.
  warn("Unable to import 'webrtcvad'. This package enables noise removal and is recommended.")


Loaded encoder "pretrained.pt" trained to step 1564501
Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder\saved_models\pretrained\pretrained.pt


In [11]:
def embedEX(in_fpath):
    filepath=r'D:\voiceset\dataset\test_wav'
#     filepath=r'D:\voiceset\dataset\wav'
    in_fpath=os.path.join(filepath,in_fpath[:7],in_fpath[8:-6],in_fpath[-5:]+'.wav')
    reprocessed_wav = encoder.preprocess_wav(in_fpath)
    original_wav, sampling_rate = librosa.load(in_fpath)
    preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
    embed,_ = encoder.embed_utterance(preprocessed_wav)
    return embed

In [5]:
class I2E_NetL1LNO(torch.nn.Module):
    def __init__(self, in_dim, n_hidden_1,n_hidden_2,out_dim):
        super(I2E_NetL1LNO, self).__init__()
        self.layer1 = torch.nn.Sequential(torch.nn.Linear(in_dim, n_hidden_1), torch.nn.ReLU(True))
        self.layer2 = torch.nn.Sequential(torch.nn.Linear(n_hidden_1, n_hidden_2), torch.nn.ReLU(True))
        self.layer3 = torch.nn.Sequential(torch.nn.Linear(n_hidden_2, out_dim))
 
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [6]:
#模型
model=I2E_NetL1LNO(400,350,300,256)
#损失函数
# criterion = torch.nn.MSELoss(reduce=True, size_average=False)
# criterion = torch.nn.MSELoss(reduction='sum')
criterion = torch.nn.L1Loss(reduction='sum')
#优化器
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
# 训练模型
epoch = 0
totaldir='D:\\pycharm_project\\vs2ttsvoice\\I2E'
#损失值写入文件

for i in range(80):
    pathstr =str(i+1)+'.txt' 
    pathstr=os.path.join(totaldir, pathstr)
    file=open( pathstr)
    for line in file.readlines():
        data=line.strip().replace('[','').replace(']','').split()
        filename=data[0]
        data=data[1:]
        out=list(map(float,data))
        out=torch.tensor(out)
        pred=model(out)
        label=embedEX(filename)
        label=torch.from_numpy(label)
        loss = criterion(pred, label)
        print_loss = loss.data.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch+=1
        if epoch%100 == 0:
        #         print('label:{0}\n,out:{1}\n,pred:{2}\n'.format(label.size(),out.size(),pred.size()))
        #保存参数、
            print('epoch: {}, loss: {:.4}'.format(epoch, loss.data.item()))
            state = { 'model':model.state_dict(), 'epoch':epoch }                       
            if not os.path.isdir('checkpoint'):
                 os.mkdir('checkpoint')    
            torch.save(state, '.\checkpoint\model2L1LNO_epoch_%d.ckpt' % (epoch))
            file_save = open('lossL1LNO.txt',mode='a')
            file_save.write('\n'+'epoch:'+str(epoch)+'  loss:'+str(loss.data.item()))
            file_save.close() 

epoch: 100, loss: 8.389
epoch: 200, loss: 6.173
epoch: 300, loss: 4.693
epoch: 400, loss: 6.618
epoch: 500, loss: 6.372
epoch: 600, loss: 5.147
epoch: 700, loss: 5.015
epoch: 800, loss: 5.04
epoch: 900, loss: 6.952
epoch: 1000, loss: 5.509
epoch: 1100, loss: 6.967
epoch: 1200, loss: 5.767
epoch: 1300, loss: 6.214
epoch: 1400, loss: 6.127
epoch: 1500, loss: 5.97
epoch: 1600, loss: 3.643
epoch: 1700, loss: 7.576
epoch: 1800, loss: 5.173
epoch: 1900, loss: 5.027
epoch: 2000, loss: 5.232
epoch: 2100, loss: 3.861
epoch: 2200, loss: 6.741
epoch: 2300, loss: 3.846
epoch: 2400, loss: 5.695
epoch: 2500, loss: 3.623
epoch: 2600, loss: 5.151
epoch: 2700, loss: 3.075
epoch: 2800, loss: 8.279
epoch: 2900, loss: 3.59
epoch: 3000, loss: 5.45
epoch: 3100, loss: 3.845
epoch: 3200, loss: 5.758
epoch: 3300, loss: 7.534
epoch: 3400, loss: 5.621
epoch: 3500, loss: 6.22
epoch: 3600, loss: 4.16
epoch: 3700, loss: 5.242
epoch: 3800, loss: 7.759
epoch: 3900, loss: 5.712
epoch: 4000, loss: 6.928
epoch: 4100, lo

epoch: 32200, loss: 7.698
epoch: 32300, loss: 5.272
epoch: 32400, loss: 4.706
epoch: 32500, loss: 6.069
epoch: 32600, loss: 4.414
epoch: 32700, loss: 4.8
epoch: 32800, loss: 3.599
epoch: 32900, loss: 4.85
epoch: 33000, loss: 6.026
epoch: 33100, loss: 3.578
epoch: 33200, loss: 8.58
epoch: 33300, loss: 5.966
epoch: 33400, loss: 5.002
epoch: 33500, loss: 4.77
epoch: 33600, loss: 6.193
epoch: 33700, loss: 6.089
epoch: 33800, loss: 4.574
epoch: 33900, loss: 5.939
epoch: 34000, loss: 3.927
epoch: 34100, loss: 3.423
epoch: 34200, loss: 4.861
epoch: 34300, loss: 6.453
epoch: 34400, loss: 7.78
epoch: 34500, loss: 6.182
epoch: 34600, loss: 4.48
epoch: 34700, loss: 5.792
epoch: 34800, loss: 6.151
epoch: 34900, loss: 5.014
epoch: 35000, loss: 5.972
epoch: 35100, loss: 6.429
epoch: 35200, loss: 5.842
epoch: 35300, loss: 4.1
epoch: 35400, loss: 6.474
epoch: 35500, loss: 4.472
epoch: 35600, loss: 4.747
epoch: 35700, loss: 5.335
epoch: 35800, loss: 4.6
epoch: 35900, loss: 3.581
epoch: 36000, loss: 3.3

epoch: 63900, loss: 3.792
epoch: 64000, loss: 7.875
epoch: 64100, loss: 3.78
epoch: 64200, loss: 6.657
epoch: 64300, loss: 5.614
epoch: 64400, loss: 3.555
epoch: 64500, loss: 3.833
epoch: 64600, loss: 6.799
epoch: 64700, loss: 3.856
epoch: 64800, loss: 4.143
epoch: 64900, loss: 5.733
epoch: 65000, loss: 5.646
epoch: 65100, loss: 6.1
epoch: 65200, loss: 6.684
epoch: 65300, loss: 3.616
epoch: 65400, loss: 7.225
epoch: 65500, loss: 5.255
epoch: 65600, loss: 7.275
epoch: 65700, loss: 4.624
epoch: 65800, loss: 4.681
epoch: 65900, loss: 2.537
epoch: 66000, loss: 6.347
epoch: 66100, loss: 4.561
epoch: 66200, loss: 5.989
epoch: 66300, loss: 6.278
epoch: 66400, loss: 5.663
epoch: 66500, loss: 4.15
epoch: 66600, loss: 6.095
epoch: 66700, loss: 4.595
epoch: 66800, loss: 4.423
epoch: 66900, loss: 6.39
epoch: 67000, loss: 6.346
epoch: 67100, loss: 5.664
epoch: 67200, loss: 5.761
epoch: 67300, loss: 6.37
epoch: 67400, loss: 5.639
epoch: 67500, loss: 4.779
epoch: 67600, loss: 4.194
epoch: 67700, loss

epoch: 95600, loss: 4.0
epoch: 95700, loss: 4.933
epoch: 95800, loss: 4.443
epoch: 95900, loss: 7.613
epoch: 96000, loss: 3.895
epoch: 96100, loss: 5.335
epoch: 96200, loss: 5.84
epoch: 96300, loss: 5.59
epoch: 96400, loss: 4.233
epoch: 96500, loss: 4.898
epoch: 96600, loss: 4.505
epoch: 96700, loss: 8.319
epoch: 96800, loss: 4.818
epoch: 96900, loss: 4.445
epoch: 97000, loss: 4.728
epoch: 97100, loss: 5.481
epoch: 97200, loss: 2.565
epoch: 97300, loss: 5.123
epoch: 97400, loss: 4.527
epoch: 97500, loss: 4.553
epoch: 97600, loss: 5.202
epoch: 97700, loss: 8.232
epoch: 97800, loss: 5.673
epoch: 97900, loss: 6.783
epoch: 98000, loss: 7.117
epoch: 98100, loss: 5.524
epoch: 98200, loss: 6.075
epoch: 98300, loss: 4.694
epoch: 98400, loss: 5.055
epoch: 98500, loss: 4.176
epoch: 98600, loss: 5.324
epoch: 98700, loss: 6.715
epoch: 98800, loss: 5.369
epoch: 98900, loss: 4.793


RuntimeError: size mismatch, m1: [1 x 360], m2: [400 x 350] at ..\aten\src\TH/generic/THTensorMath.cpp:41

In [9]:
# 读取训练参数
# checkpoint = torch.load(r'C:\NLP\tts_rtvc\rtvc\checkpoint\model3_epoch_56300.ckpt')
checkpoint = torch.load(r'D:\pycharm_project\vs2ttsvoice\checkpoint\model2L1LNO_epoch_98800.ckpt')
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [12]:
#合成语音
from IPython.utils import io
epoch=0
text="This is being said in my own voice.The computer has learned to do an impression of me."
file=open('.\I2E\ivectortest.txt')
filenamelabel=''
for line in file.readlines():
# line=file.readline()
    data=line.strip().replace('[','').replace(']','').split()
    filename=data[0]
    if (filenamelabel==filename[:7]):
        continue
    print(filename)
    data=data[1:]
#     out=[[list(map(float,data))]]
#     out=torch.tensor(out)
#     out=out.permute(0,2,1)
#     pred=model(out)
#     pred=pred.squeeze(0)
#     pred=pred.tolist()
#     print("pred",pred)
    out=list(map(float,data))
    out=torch.tensor(out)
    pred=model(out)
    pred=pred.tolist()
    print("pred",pred)
    print("Synthesizing 预测语音...")
    with io.capture_output() as captured:
      specs = synthesizer.synthesize_spectrograms([text], [pred])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
#     sf.write(filename[:7]+"A.wav", generated_wav, synthesizer.sample_rate)
    epoch =+1
    #原始声音合成
    print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
    label=embedEX(filename)
    print("Synthesizing 原始语音...")
    print('阈值：',np.linalg.norm(pred-label))
    with io.capture_output() as captured:
       specs = synthesizer.synthesize_spectrograms([text], [label])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
    filenamelabel=filename[:7]
#     sf.write(filename[:7]+".wav", generated_wav, synthesizer.sample_rate)

id10270-5r0dWxy17C8-00001
pred [0.0001846094528445974, 0.004644153639674187, 8.404218533542007e-05, 0.0003202589869033545, -0.0010770722292363644, 0.026316214352846146, 0.0001687508774921298, 0.0484752394258976, 0.028907742351293564, 0.006552652921527624, 0.1174875870347023, 0.07460024952888489, 0.0334060974419117, 0.0007901790086179972, 0.03258991986513138, -0.0001028827391564846, 0.08424005657434464, 0.016904644668102264, -0.00021513085812330246, 0.012177280150353909, 0.009121524170041084, 0.056920334696769714, 3.487661888357252e-05, 0.00018792595074046403, 0.000578413309995085, 0.022081661969423294, 0.010066276416182518, 0.0060510761104524136, -0.000364141131285578, 0.0006397251272574067, 0.08196447044610977, 0.03569639474153519, 0.030874531716108322, 0.006002660840749741, 0.001814111601561308, 0.11624263226985931, 0.04224842041730881, 0.016528474166989326, 0.00024697979097254574, 0.001318631344474852, 1.3535944162867963e-05, 0.015906820073723793, 0.03992471843957901, 0.019500235095

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.6868262384978921


id10271-1gtz-CUIygI-00001
pred [0.0001846094528445974, 0.004644153639674187, 8.404218533542007e-05, 0.0003202589869033545, -0.0010770722292363644, 0.026316214352846146, 0.0001687508774921298, 0.0484752394258976, 0.028907742351293564, 0.006552652921527624, 0.1174875870347023, 0.07460024952888489, 0.0334060974419117, 0.0007901790086179972, 0.03258991986513138, -0.0001028827391564846, 0.08424005657434464, 0.016904644668102264, -0.00021513085812330246, 0.012177280150353909, 0.009121524170041084, 0.056920334696769714, 3.487661888357252e-05, 0.00018792595074046403, 0.000578413309995085, 0.022081661969423294, 0.010066276416182518, 0.0060510761104524136, -0.000364141131285578, 0.0006397251272574067, 0.08196447044610977, 0.03569639474153519, 0.030874531716108322, 0.006002660840749741, 0.001814111601561308, 0.11624263226985931, 0.04224842041730881, 0.016528474166989326, 0.00024697979097254574, 0.001318631344474852, 1.3535944162867963e-05, 0.015906820073723793, 0.03992471843957901, 0.019500235095

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.7752732131705725


id10272-0qqQJVFKpbA-00001
pred [0.0001846094528445974, 0.004644153639674187, 8.404218533542007e-05, 0.0003202589869033545, -0.0010770722292363644, 0.026316214352846146, 0.0001687508774921298, 0.0484752394258976, 0.028907742351293564, 0.006552652921527624, 0.1174875870347023, 0.07460024952888489, 0.0334060974419117, 0.0007901790086179972, 0.03258991986513138, -0.0001028827391564846, 0.08424005657434464, 0.016904644668102264, -0.00021513085812330246, 0.012177280150353909, 0.009121524170041084, 0.056920334696769714, 3.487661888357252e-05, 0.00018792595074046403, 0.000578413309995085, 0.022081661969423294, 0.010066276416182518, 0.0060510761104524136, -0.000364141131285578, 0.0006397251272574067, 0.08196447044610977, 0.03569639474153519, 0.030874531716108322, 0.006002660840749741, 0.001814111601561308, 0.11624263226985931, 0.04224842041730881, 0.016528474166989326, 0.00024697979097254574, 0.001318631344474852, 1.3535944162867963e-05, 0.015906820073723793, 0.03992471843957901, 0.019500235095

...

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.7468743505106755


id10276-3vWez3baO60-00001
pred [0.0001846094528445974, 0.004644153639674187, 8.404218533542007e-05, 0.0003202589869033545, -0.0010770722292363644, 0.026316214352846146, 0.0001687508774921298, 0.0484752394258976, 0.028907742351293564, 0.006552652921527624, 0.1174875870347023, 0.07460024952888489, 0.0334060974419117, 0.0007901790086179972, 0.03258991986513138, -0.0001028827391564846, 0.08424005657434464, 0.016904644668102264, -0.00021513085812330246, 0.012177280150353909, 0.009121524170041084, 0.056920334696769714, 3.487661888357252e-05, 0.00018792595074046403, 0.000578413309995085, 0.022081661969423294, 0.010066276416182518, 0.0060510761104524136, -0.000364141131285578, 0.0006397251272574067, 0.08196447044610977, 0.03569639474153519, 0.030874531716108322, 0.006002660840749741, 0.001814111601561308, 0.11624263226985931, 0.04224842041730881, 0.016528474166989326, 0.00024697979097254574, 0.001318631344474852, 1.3535944162867963e-05, 0.015906820073723793, 0.03992471843957901, 0.019500235095

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.742175592072854


id10277-0rpfN7wThsg-00001
pred [0.0001846094528445974, 0.004644153639674187, 8.404218533542007e-05, 0.0003202589869033545, -0.0010770722292363644, 0.026316214352846146, 0.0001687508774921298, 0.0484752394258976, 0.028907742351293564, 0.006552652921527624, 0.1174875870347023, 0.07460024952888489, 0.0334060974419117, 0.0007901790086179972, 0.03258991986513138, -0.0001028827391564846, 0.08424005657434464, 0.016904644668102264, -0.00021513085812330246, 0.012177280150353909, 0.009121524170041084, 0.056920334696769714, 3.487661888357252e-05, 0.00018792595074046403, 0.000578413309995085, 0.022081661969423294, 0.010066276416182518, 0.0060510761104524136, -0.000364141131285578, 0.0006397251272574067, 0.08196447044610977, 0.03569639474153519, 0.030874531716108322, 0.006002660840749741, 0.001814111601561308, 0.11624263226985931, 0.04224842041730881, 0.016528474166989326, 0.00024697979097254574, 0.001318631344474852, 1.3535944162867963e-05, 0.015906820073723793, 0.03992471843957901, 0.019500235095

KeyboardInterrupt: 